<a href="https://colab.research.google.com/github/ahmededrisahmed/fmcs39130/blob/main/lstm_model_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io
import re
import os
import numpy as np
from numpy import array
import string
import time
import random
import math
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import seaborn as sns
import sklearn
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, f1_score, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import keras
from keras import layers
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from IPython.display import display
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
random.seed(10)

nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import files
uploaded = files.upload()

Saving amazon_alexa.tsv to amazon_alexa.tsv


In [4]:
df = pd.read_csv('amazon_alexa.tsv', sep='\\t')
df = df[['verified_reviews', 'feedback']]

df = df[df.verified_reviews != "Neutral"]
df['verified_reviews'] = df['verified_reviews'].apply(lambda x: x.lower())
df['verified_reviews'] = df['verified_reviews'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(df[ df['feedback'] == 1 ].size)
print(df[ df['feedback'] == 0 ].size)

df.columns = ["verified_reviews", "feedback"]
x = df['verified_reviews'].values
y = df['feedback'].values

x_train, x_test, y_train, y_test = \
train_test_split(x, y, test_size=0.1, random_state=123)

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test)
vocab_size=len(tokenizer.word_index)+1

maxlen=10
xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

5786
514


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
embedding_dim=50
m  =Sequential()
m.add(layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=maxlen))
m.add(layers.LSTM(units=50,return_sequences=True))
m.add(layers.LSTM(units=10))
m.add(layers.Dropout(0.5))
m.add(layers.Dense(8))
m.add(layers.Dense(1, activation="sigmoid"))
m.compile(optimizer="adam", loss="binary_crossentropy",metrics=['accuracy'])
m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 50)            225750    
                                                                 
 lstm (LSTM)                 (None, 10, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 10)                2440      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 248,487
Trainable params: 248,487
Non-trai

In [6]:
history = m.fit(xtrain,y_train,epochs=100, batch_size= 32, verbose=2)

Epoch 1/100
89/89 - 8s - loss: 0.3861 - accuracy: 0.9069 - 8s/epoch - 91ms/step
Epoch 2/100
89/89 - 2s - loss: 0.2762 - accuracy: 0.9178 - 2s/epoch - 18ms/step
Epoch 3/100
89/89 - 2s - loss: 0.2394 - accuracy: 0.9192 - 2s/epoch - 17ms/step
Epoch 4/100
89/89 - 2s - loss: 0.2317 - accuracy: 0.9189 - 2s/epoch - 18ms/step
Epoch 5/100
89/89 - 1s - loss: 0.2226 - accuracy: 0.9178 - 1s/epoch - 17ms/step
Epoch 6/100
89/89 - 2s - loss: 0.2227 - accuracy: 0.9182 - 2s/epoch - 18ms/step
Epoch 7/100
89/89 - 2s - loss: 0.2211 - accuracy: 0.9171 - 2s/epoch - 18ms/step
Epoch 8/100
89/89 - 2s - loss: 0.2136 - accuracy: 0.9182 - 2s/epoch - 18ms/step
Epoch 9/100
89/89 - 2s - loss: 0.2124 - accuracy: 0.9185 - 2s/epoch - 18ms/step
Epoch 10/100
89/89 - 2s - loss: 0.2138 - accuracy: 0.9178 - 2s/epoch - 19ms/step
Epoch 11/100
89/89 - 2s - loss: 0.2056 - accuracy: 0.9171 - 2s/epoch - 20ms/step
Epoch 12/100
89/89 - 2s - loss: 0.2030 - accuracy: 0.9168 - 2s/epoch - 20ms/step
Epoch 13/100
89/89 - 2s - loss: 0.201

In [7]:
loss, acc = m.evaluate(xtrain, y_train, verbose=2)
print("Training Accuracy: ", acc)
loss, acc = m.evaluate(xtest, y_test, verbose=2)
print("Test Accuracy: ", acc)

89/89 - 2s - loss: 0.0491 - accuracy: 0.9841 - 2s/epoch - 19ms/step
Training Accuracy:  0.9841269850730896
10/10 - 0s - loss: 0.5587 - accuracy: 0.9238 - 70ms/epoch - 7ms/step
Test Accuracy:  0.9238095283508301


In [8]:
m.save('lstm_m.h5')
# load m from single file
mm = load_model('lstm_m.h5')
# make predictions

In [17]:

# import modules
import pandas as pd
import tweepy

# function to display data of each tweet
def printtweetdata(n, ith_tweet):
		print()
		print(f"Tweet {n}:")
		print(f"Username:{ith_tweet[0]}")
		print(f"Description:{ith_tweet[1]}")
		print(f"Location:{ith_tweet[2]}")
		print(f"Following Count:{ith_tweet[3]}")
		print(f"Follower Count:{ith_tweet[4]}")
		print(f"Total Tweets:{ith_tweet[5]}")
		print(f"Retweet Count:{ith_tweet[6]}")
		print(f"Tweet Text:{ith_tweet[7]}")
		print(f"Hashtags Used:{ith_tweet[8]}")


# function to perform data extraction
def scrape(words, date_since, numtweet):

		# Creating DataFrame using pandas
		db = pd.DataFrame(columns=['username',
								'description',
								'location',
								'following',
								'followers',
								'totaltweets',
								'retweetcount',
								'text',
								'hashtags'])

		# We are using .Cursor() to search
		# through twitter for the required tweets.
		# The number of tweets can be
		# restricted using .items(number of tweets)
		tweets = tweepy.Cursor(api.search,
							words, lang="en",
							since_id=date_since,
							tweet_mode='extended').items(numtweet)


		# .Cursor() returns an iterable object. Each item in
		# the iterator has various attributes
		# that you can access to
		# get information about each tweet
		list_tweets = [tweet for tweet in tweets]

		# Counter to maintain Tweet Count
		i = 1

		# we will iterate over each tweet in the
		# list for extracting information about each tweet
		for tweet in list_tweets:
				username = tweet.user.screen_name
				description = tweet.user.description
				location = tweet.user.location
				following = tweet.user.friends_count
				followers = tweet.user.followers_count
				totaltweets = tweet.user.statuses_count
				retweetcount = tweet.retweet_count
				hashtags = tweet.entities['hashtags']

				# Retweets can be distinguished by
				# a retweeted_status attribute,
				# in case it is an invalid reference,
				# except block will be executed
				try:
						text = tweet.retweeted_status.full_text
				except AttributeError:
						text = tweet.full_text
				hashtext = list()
				for j in range(0, len(hashtags)):
						hashtext.append(hashtags[j]['text'])

				# Here we are appending all the
				# extracted information in the DataFrame
				ith_tweet = [username, description,
							location, following,
							followers, totaltweets,
							retweetcount, text, hashtext]
				db.loc[len(db)] = ith_tweet

				# Function call to print tweet data on screen
				printtweetdata(i, ith_tweet)
				i = i+1
		filename = 'tweets.csv'

		# we will save our database as a CSV file.
		db.to_csv(filename)

if __name__ == '__main__':

		# Enter your own credentials obtained
		# from your developer account
		consumer_key = "sDq4iViNrUs01nRtozqjlFMXC"
		consumer_secret = "AhMHrrq8qYt4aAueIKiwxjLI2SRHZ4vpI84UMXS8IEojpdCsxt"
		access_key = "1588248654548279301-RglNooFZz3LgM5BLCmtNLN0mdS1wpz"
		access_secret = "uUx5tsnNqgb3gyFqhLaUjlEAE3YCUYl4rLPQVssYkvEUJ"


		auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
		auth.set_access_token(access_key, access_secret)
		api = tweepy.API(auth)

		# Enter Hashtag and initial date
		print("Enter Twitter HashTag to search for")
		words = input()
		print("Enter Date since The Tweets are required in yyyy-mm--dd")
		date_since = input()

		# number of tweets you want to extract in one run
		numtweet = 1000
		scrape(words, date_since, numtweet)
  # amazonalexa
		print('Scraping has completed!')

Enter Twitter HashTag to search for
amazonalexa
Enter Date since The Tweets are required in yyyy-mm--dd
2019-01-01

Tweet 1:
Username:ThomasBurdick13
Description:
Location:
Following Count:2
Follower Count:0
Total Tweets:25
Retweet Count:0
Tweet Text:Purchase Google Voice Accounts

#googlevoice #voicetechnology #voicefirst #alexa #amazonalexa #voiceassistants 

If You Want To Buy Verified Google Voice Accounts.

Please Visit Our Website: https://t.co/qEcZP7HU02 https://t.co/gGWmPfeWCg
Hashtags Used:['googlevoice', 'voicetechnology', 'voicefirst', 'alexa', 'amazonalexa', 'voiceassistants']

Tweet 2:
Username:PoojaraTelecom
Description:Poojara Telecom is Gujarat's Most Trusted & Recommended Mobile & Smart Electronic Retail Chain with 200+ stores across Gujarat.

Since 1994
80L + Happy Customer
Location:Gujarat (India)
Following Count:30
Follower Count:1108
Total Tweets:3375
Retweet Count:0
Tweet Text:Poojara 𝗖𝗛𝗥𝗜𝗦𝗧𝗠𝗔𝗦 𝗖𝗔𝗥𝗡𝗜𝗩𝗔𝗟🎄is Live!🎅
✅An exciting #Buy1Get1 Free offer on #AmazonEchoDot

In [ ]:
new_data = pd.read_csv('tweets.csv')
new_data = new_data[['text']]
new_data

In [22]:
corpus = [] 
for i in range(len(new_data)):
  s = re.sub('[^a-zA-Z]', " ", new_data['text'][i])
  s = s.lower()
  s = s.split()
  s = [word for word in s if word not in stopwords.words('english')]
  s = ' '.join(s)
  ps = PorterStemmer()
  s = ps.stem(s)
  corpus.append(s)
corpus

pre_pro=corpus
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(pre_pro)
new_x= tokenizer.texts_to_sequences(pre_pro)
vocab_size=len(tokenizer.word_index)+1
new_x=pad_sequences(new_x,padding='post', maxlen=10)

In [23]:
yhat = mm.predict(new_x, verbose=0)

In [24]:
s = []
for i in yhat:
    lable = int(i)
    if lable == 1:
        s.append(1)
    else:
        s.append(0)

new_data['feedback'] = s

new_data

,text,feedback
0,Purchase Google Voice Accounts\n\n#googlevoice...,0
1,Poojara 𝗖𝗛𝗥𝗜𝗦𝗧𝗠𝗔𝗦 𝗖𝗔𝗥𝗡𝗜𝗩𝗔𝗟🎄is Live!🎅\n✅An exci...,0
2,Personalize your user's experience with conver...,0
3,Get to bed early because Santa is COMING!!\n\n...,0
4,What a brilliant advertisement 😂👌 #Alexa #amaz...,0
...,...,...
78,I created my own @alexa99 skill with no coding...,0
79,#alexa out here trolling me!! #amazonalexa htt...,0
80,We create exclusive and custom skills for busi...,0
81,Amazon Alexa: The 1990s called and wanted thei...,0
